# Ollama Model Server

### Install Ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Check Installation

In [ ]:
!ollama --version

### Run Ollama Server

In [54]:
import subprocess
import threading
import time
import requests
import json
import os

# Function to run Ollama in background
def run_ollama():
    # Set Ollama to listen on all interfaces
    env_vars = os.environ.copy()
    env_vars["OLLAMA_HOST"] = "0.0.0.0"
    env_vars["PATH"] = "/usr/local/bin:" + env_vars.get("PATH", "")

    # Use subprocess.Popen for a non-blocking background process
    # Redirect stdout and stderr to DEVNULL to keep the console clean
    subprocess.Popen(["/usr/local/bin/ollama", "serve"], env=env_vars, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Start Ollama in a separate thread
ollama_thread = threading.Thread(target=run_ollama, daemon=True)
ollama_thread.start()

# Wait for Ollama to start and become responsive
ollama_url = "http://127.0.0.1:11434"
max_retries = 60 # Increased retries (60 * 2 seconds = 120 seconds total wait)
for i in range(max_retries):
    try:
        response = requests.get(f"{ollama_url}/api/tags", timeout=1)
        if response.status_code == 200:
            print("Ollama server started and responsive.")
            break
    except requests.exceptions.ConnectionError:
        pass
    print(f"Waiting for Ollama server... ({i+1}/{max_retries})")
    time.sleep(2)
else:
    print("Ollama server did not become responsive in time.")

Waiting for Ollama server... (1/60)
Ollama server started and responsive.


In [ ]:
!ps aux | grep ollama

### Pull required ollama model

In [ ]:
# !ollama pull llama3.1:8b
!ollama pull mistral:7b


### Test model

In [49]:
!ollama run llama3.2:7b "Explain RAG in one paragraph"


>>> Send a message (/? for help)y
 I'm glad to assist you! How can I help you today?

If you have any questions or need information about a specific topic, feel free to ask. I'll do my best to provide helpful and informative responses.

Here are some examples of topics that I can help with:

* General knowledge (e.g., science, history, arts, literature)
* Mathematics problems
* Programming questions
* Explanations of technical concepts
* Writing suggestions or edits
* Suggestions for books, movies, or music based on your preferences
* Recipes and cooking tips
* Health advice (within reason; I'm not a doctor, but I can provide general information about nutrition, exercise, and wellness)

I look forward to helping you with whatever you need! Just let me know.

>>> Send a message (/? for help)^C


In [57]:
!ss -tulnp | grep 11434


tcp   LISTEN 0      4096               *:11434            *:*    users:(("ollama",pid=40834,fd=3))      


In [ ]:
# Download the latest Cloudflare Tunnel (cloudflared) binary for Linux
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64

# Make the downloaded cloudflared binary executable
!chmod +x cloudflared-linux-amd64

In [ ]:
# Start a Cloudflare tunnel that exposes the local Ollama server
# running on port 11434 to a public HTTPS URL
!./cloudflared-linux-amd64 tunnel --url http://localhost:11434
